In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
from os import path

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

## Load data
Load interpolated/point data for vessels/whales - this is at 10 minute resolution, allowing detection of encounters between vessels/whales

In [ ]:
from data_processing import load_data, encounters
from data_processing.snaptocoast import internal_points_to_coast

In [ ]:
# base data for drawing map
whales, vessels, protected_areas, basemap, bounds = load_data.load_all()

In [ ]:
# Load whale lines for plotting
whales_lines = load_data.load_whale_lines('data/whales/df_all_3.csv', 2193)

In [ ]:
# Vessel points, interpolated to 10 minutes, for calculating encounters
vessel_data_files = [path.join('data', 'vessels', f'{vessel_type}_points.gpkg') for vessel_type in ['Fishing', 'Other', 'Cargo', 'Passenger', 'Tanker']]
vessel_data_sets = [load_data.load_vessel_points(vdf, 2193) for vdf in vessel_data_files]

vessel_points = pd.concat(vessel_data_sets)

vessel_points['geometry'] = vessel_points['geometry'].progress_apply(internal_points_to_coast, coasts=basemap)

In [ ]:
# Whale points, interpolated to 10 minutes, for calculating encounters
whales_points = load_data.load_whales('data/whales/df_all_3.csv', vessel_points.geometry.total_bounds, 2193, 10)

In [ ]:
# Calculate encounters from vessel perspective (takes 1+ hours)
vessel_encounters_fname = 'vessel_encounters.pickle'
if path.isfile(vessel_encounters_fname):
    vessel_encounters = pd.read_pickle(vessel_encounters_fname)
else:
    vessel_encounters = vessel_points.copy()
    vessel_encounters['whale_dist'] = encounters.dist_to_whales(vessel_points, whales_points)
    vessel_encounters.to_pickle(vessel_encounters_fname)

In [ ]:
# Calculate encounters from whale perspective (takes 1+ hours)
whale_encounters_fname = 'whale_encounters.pickle'
if path.isfile(whale_encounters_fname):
    whales_encounters = pd.read_pickle(whale_encounters_fname)
else:
    whales_encounters = whales_points.copy()
    whales_encounters['dist_to_vessel'] = encounters.dist_to_whales(whales_points, vessel_points)
    whales_encounters.to_pickle(whale_encounters_fname)

In [ ]:
# Simplify geometry & drop columns to reduce html size
whales_lines.drop(columns=['id'], inplace=True)
vessel_points.drop(columns=['timestamp', 'callsign', 'type'], inplace=True)

## Plotting
Plot vessel + whale traces, color encounters

In [ ]:
from plotting import heatmap

In [ ]:
from bokeh.io import output_notebook, output_file, export_png
from bokeh.plotting import show
output_notebook()
output_file('encounters.html', title='Encounters')

In [ ]:
p = heatmap.encounters_map(whales_lines, vessels, vessel_encounters, protected_areas, basemap, bounds, max_dist=20000)
show(p)

In [ ]:
export_png(p, filename='encounters.png')